In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Modified to remove English stopwords

In [ ]:
import pandas as pd
import numpy as np

file_path = '{Insert file path here}'
df = pd.read_excel(file_path)

# Check for unintended data and print the exact row
unintended_indices = df[(df['v1'] != 'ham') & (df['v1'] != 'spam')].index
if len(unintended_indices) > 0:
    print("UNINTENDED DATA found at rows:")
    print(df.loc[unintended_indices])

import pandas as pd
import numpy as np
import nltk
import re
import nltk
import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import GridSearchCV

nltk.download('stopwords')

df = df.replace(['ham', 'spam'], [0, 1])

df['Count'] = 0

for i in np.arange(0, len(df['v2'])):
    df.loc[i, 'Count'] = len(str(df.loc[i, 'v2']))

corpus = []
ps = PorterStemmer()

for i in range(len(df)):
    msg = df['v2'][i]
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(df['v2'][i]))
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', str(df['v2'][i]))
    msg = re.sub('£|\$', 'moneysymb', str(df['v2'][i]))
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', str(df['v2'][i]))
    msg = re.sub('\d+(\.\d+)?', 'numbr', str(df['v2'][i]))

    msg = re.sub('[^\w\d\s]', ' ', str(df['v2'][i]))

    msg = msg.lower()
    msg = msg.split()

    msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]

    msg = ' '.join(msg)
    corpus.append(msg)

cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

y = df['v1']
y = y.astype(str)
le = LabelEncoder()
y = le.fit_transform(y)

x = pd.DataFrame(x)
y = pd.DataFrame(y)

common_indices = x.index.intersection(y.index)
x = x.loc[common_indices, :]
y = y.loc[common_indices]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=0)

param_grid_dt = {
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3],
    'criterion': ['gini'],
    'max_features': ['sqrt']
}

start_time_dt = time.time()

# Decision Tree
dt = DecisionTreeClassifier(random_state=50)
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, n_jobs=-1, verbose=1)
grid_search_dt.fit(xtrain, ytrain)

best_dt_params = grid_search_dt.best_params_
print(f"Best Decision Tree Parameters: {best_dt_params}")

dt.fit(xtrain, ytrain)

y_pred_dt = dt.predict(xtest)

end_time_dt = time.time()

execution_time_dt = end_time_dt - start_time_dt
execution_time_dt_rounded = round(execution_time_dt)

cm_dt = confusion_matrix(ytest, y_pred_dt)

print("Decision Tree Confusion Matrix:")
print(cm_dt)

print("Execution time for Decision Tree:", execution_time_dt, "seconds")

print("Decision Tree Accuracy : %0.5f \n\n" % accuracy_score(ytest, dt.predict(xtest)))
print("Decision Tree Classification Report:")
print(classification_report(ytest, dt.predict(xtest)))

param_grid_nb = {
    'alpha': [1, 2, 5]
}

start_time_nb = time.time()

# Multinomial Naive Bayes
nb = MultinomialNB()

grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, n_jobs=-1, verbose=1)
grid_search_nb.fit(xtrain, ytrain)

best_nb_params = grid_search_nb.best_params_
print(f"Best Naive Bayes Parameters: {best_nb_params}")

nb.fit(xtrain, ytrain)

y_pred_nb = nb.predict(xtest)

end_time_nb = time.time()

execution_time_nb = end_time_nb - start_time_nb

cm_nb = confusion_matrix(ytest, y_pred_nb)

print("\nMultinomial Naive Bayes Confusion Matrix:")
print(cm_nb)

print("Execution time for Multinomial Naive Bayes:", execution_time_nb, "seconds")
execution_time_nb_rounded = round(execution_time_nb)

print("Multinomial Naive Bayes Accuracy : %0.5f \n\n" % accuracy_score(ytest, nb.predict(xtest)))
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(ytest, nb.predict(xtest)))

Modified to remove Turkish stopwords (Dataset 2)

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from snowballstemmer import TurkishStemmer

nltk.download('stopwords')

file_path = '{Insert file path here}'
df = pd.read_excel(file_path)

unintended_indices = df[(df['v1'] != 'ham') & (df['v1'] != 'spam')].index
if len(unintended_indices) > 0:
    print("UNINTENDED DATA found at rows:")
    print(df.loc[unintended_indices])

df = df.replace(['ham', 'spam'], [0, 1])

df['Count'] = 0

for i in np.arange(0, len(df['v2'])):
    df.loc[i, 'Count'] = len(str(df.loc[i, 'v2']))

corpus = []
stemmer = TurkishStemmer()

for i in range(len(df)):
    msg = df['v2'][i]
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(df['v2'][i]))
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', str(df['v2'][i]))
    msg = re.sub('£|\$', 'moneysymb', str(df['v2'][i]))
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', str(df['v2'][i]))
    msg = re.sub('\d+(\.\d+)?', 'numbr', str(df['v2'][i]))

    msg = re.sub('[^\w\d\s]', ' ', str(df['v2'][i]))

    msg = msg.lower()
    msg = msg.split()

    msg = [stemmer.stemWord(word) for word in msg if not word in set(stopwords.words('turkish'))]

    msg = ' '.join(msg)
    corpus.append(msg)

cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

y = df['v1']
y = y.astype(str)
le = LabelEncoder()
y = le.fit_transform(y)

x = pd.DataFrame(x)
y = pd.DataFrame(y)

common_indices = x.index.intersection(y.index)
x = x.loc[common_indices, :]
y = y.loc[common_indices]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=0)

param_grid_dt = {
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3],
    'criterion': ['gini'],
    'max_features': ['sqrt']
}

start_time_dt = time.time()

dt = DecisionTreeClassifier(random_state=50)
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, n_jobs=-1, verbose=1)
grid_search_dt.fit(xtrain, ytrain)

best_dt_params = grid_search_dt.best_params_
print(f"Best Decision Tree Parameters: {best_dt_params}")

dt.fit(xtrain, ytrain)

y_pred_dt = dt.predict(xtest)

end_time_dt = time.time()

execution_time_dt = end_time_dt - start_time_dt
execution_time_dt_rounded = round(execution_time_dt)

cm_dt = confusion_matrix(ytest, y_pred_dt)

print("Decision Tree Confusion Matrix:")
print(cm_dt)

print("Execution time for Decision Tree:", execution_time_dt, "seconds")

print("Decision Tree Accuracy : %0.5f \n\n" % accuracy_score(ytest, dt.predict(xtest)))
print("Decision Tree Classification Report:")
print(classification_report(ytest, dt.predict(xtest)))

param_grid_nb = {
    'alpha': [1, 2, 5]
}

start_time_nb = time.time()

nb = MultinomialNB()

grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, n_jobs=-1, verbose=1)
grid_search_nb.fit(xtrain, ytrain)

best_nb_params = grid_search_nb.best_params_
print(f"Best Naive Bayes Parameters: {best_nb_params}")

nb.fit(xtrain, ytrain)

y_pred_nb = nb.predict(xtest)

end_time_nb = time.time()

execution_time_nb = end_time_nb - start_time_nb

cm_nb = confusion_matrix(ytest, y_pred_nb)

print("\nMultinomial Naive Bayes Confusion Matrix:")
print(cm_nb)

print("Execution time for Multinomial Naive Bayes:", execution_time_nb, "seconds")
execution_time_nb_rounded = round(execution_time_nb)

print("Multinomial Naive Bayes Accuracy : %0.5f \n\n" % accuracy_score(ytest, nb.predict(xtest)))
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(ytest, nb.predict(xtest)))

Modified to remove Bengali stopwords (Dataset 4)

In [ ]:
pip install bnlp_toolkit

In [ ]:
import pandas as pd
import numpy as np
import re
import time
import bnlp
from bnlp import NLTKTokenizer
from bnlp import BengaliCorpus

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

file_path = '{Insert file path here}'
df = pd.read_excel(file_path)

unintended_indices = df[(df['v1'] != 'ham') & (df['v1'] != 'spam')].index
if len(unintended_indices) > 0:
    print("UNINTENDED DATA found at rows:")
    print(df.loc[unintended_indices])

df = df.replace(['ham', 'spam'], [0, 1])

df['Count'] = 0

for i in np.arange(0, len(df['v2'])):
    df.loc[i, 'Count'] = len(str(df.loc[i, 'v2']))

bengali_corpus = BengaliCorpus()

corpus = []
bnltk = NLTKTokenizer()
stopwords_bn = bengali_corpus.stopwords

for i in range(len(df)):
    msg = df['v2'][i]
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(df['v2'][i]))
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', str(df['v2'][i]))
    msg = re.sub('£|\$', 'moneysymb', str(df['v2'][i]))
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', str(df['v2'][i]))
    msg = re.sub('\d+(\.\d+)?', 'numbr', str(df['v2'][i]))

    msg = re.sub('[^\w\d\s]', ' ', str(df['v2'][i]))

    msg = msg.lower()
    msg = msg.split()

    msg = [word for word in msg if word not in stopwords_bn]

    msg = ' '.join(msg)
    corpus.append(msg)

cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

y = df['v1']
y = y.astype(str)
le = LabelEncoder()
y = le.fit_transform(y)

x = pd.DataFrame(x)
y = pd.DataFrame(y)

common_indices = x.index.intersection(y.index)
x = x.loc[common_indices, :]
y = y.loc[common_indices]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=0)

param_grid_dt = {
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3],
    'criterion': ['gini'],
    'max_features': ['sqrt']
}

start_time_dt = time.time()

dt = DecisionTreeClassifier(random_state=50)
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, n_jobs=-1, verbose=1)
grid_search_dt.fit(xtrain, ytrain)

best_dt_params = grid_search_dt.best_params_
print(f"Best Decision Tree Parameters: {best_dt_params}")

dt.fit(xtrain, ytrain)

y_pred_dt = dt.predict(xtest)

end_time_dt = time.time()

execution_time_dt = end_time_dt - start_time_dt
execution_time_dt_rounded = round(execution_time_dt)

cm_dt = confusion_matrix(ytest, y_pred_dt)

print("Decision Tree Confusion Matrix:")
print(cm_dt)

print("Execution time for Decision Tree:", execution_time_dt, "seconds")

print("Decision Tree Accuracy : %0.5f \n\n" % accuracy_score(ytest, dt.predict(xtest)))
print("Decision Tree Classification Report:")
print(classification_report(ytest, dt.predict(xtest)))

param_grid_nb = {
    'alpha': [1, 2, 5]
}

start_time_nb = time.time()

nb = MultinomialNB()

grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, n_jobs=-1, verbose=1)
grid_search_nb.fit(xtrain, ytrain)


best_nb_params = grid_search_nb.best_params_
print(f"Best Naive Bayes Parameters: {best_nb_params}")

nb.fit(xtrain, ytrain)

y_pred_nb = nb.predict(xtest)

end_time_nb = time.time()

execution_time_nb = end_time_nb - start_time_nb

cm_nb = confusion_matrix(ytest, y_pred_nb)

print("\nMultinomial Naive Bayes Confusion Matrix:")
print(cm_nb)

print("Execution time for Multinomial Naive Bayes:", execution_time_nb, "seconds")
execution_time_nb_rounded = round(execution_time_nb)

print("Multinomial Naive Bayes Accuracy : %0.5f \n\n" % accuracy_score(ytest, nb.predict(xtest)))
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(ytest, nb.predict(xtest)))

Modified to remove Indonesian stopwords (Dataset 8)

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import time

nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import GridSearchCV

file_path = '{Insert file path here}'
df = pd.read_excel(file_path)

unintended_indices = df[(df['v1'] != 'ham') & (df['v1'] != 'spam')].index
if len(unintended_indices) > 0:
    print("UNINTENDED DATA found at rows:")
    print(df.loc[unintended_indices])

df = df.replace(['ham', 'spam'], [0, 1])

df['Count'] = 0

for i in np.arange(0, len(df['v2'])):
    df.loc[i, 'Count'] = len(str(df.loc[i, 'v2']))

corpus = []
stop_words = set(stopwords.words('indonesian'))

for i in range(len(df)):
    msg = df['v2'][i]
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(df['v2'][i]))
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', str(df['v2'][i]))
    msg = re.sub('£|\$', 'moneysymb', str(df['v2'][i]))
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', str(df['v2'][i]))
    msg = re.sub('\d+(\.\d+)?', 'numbr', str(df['v2'][i]))

    msg = re.sub('[^\w\d\s]', ' ', str(df['v2'][i]))

    msg = msg.lower()
    msg = msg.split()

    msg = [word for word in msg if word not in stop_words]

    msg = ' '.join(msg)
    corpus.append(msg)

cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

y = df['v1']
y = y.astype(str)
le = LabelEncoder()
y = le.fit_transform(y)

x = pd.DataFrame(x)
y = pd.DataFrame(y)

common_indices = x.index.intersection(y.index)
x = x.loc[common_indices, :]
y = y.loc[common_indices]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=0)

param_grid_dt = {
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3],
    'criterion': ['gini'],
    'max_features': ['sqrt']
}

start_time_dt = time.time()

dt = DecisionTreeClassifier(random_state=50)
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, n_jobs=-1, verbose=1)
grid_search_dt.fit(xtrain, ytrain)

best_dt_params = grid_search_dt.best_params_
print(f"Best Decision Tree Parameters: {best_dt_params}")

dt.fit(xtrain, ytrain)

y_pred_dt = dt.predict(xtest)

end_time_dt = time.time()

execution_time_dt = end_time_dt - start_time_dt
execution_time_dt_rounded = round(execution_time_dt)

cm_dt = confusion_matrix(ytest, y_pred_dt)

print("Decision Tree Confusion Matrix:")
print(cm_dt)

print("Execution time for Decision Tree:", execution_time_dt, "seconds")

print("Decision Tree Accuracy : %0.5f \n\n" % accuracy_score(ytest, dt.predict(xtest)))
print("Decision Tree Classification Report:")
print(classification_report(ytest, dt.predict(xtest)))

param_grid_nb = {
    'alpha': [1, 2, 5]
}

start_time_nb = time.time()

nb = MultinomialNB()

grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, n_jobs=-1, verbose=1)
grid_search_nb.fit(xtrain, ytrain)

best_nb_params = grid_search_nb.best_params_
print(f"Best Naive Bayes Parameters: {best_nb_params}")

nb.fit(xtrain, ytrain)

y_pred_nb = nb.predict(xtest)

end_time_nb = time.time()

execution_time_nb = end_time_nb - start_time_nb

cm_nb = confusion_matrix(ytest, y_pred_nb)

print("\nMultinomial Naive Bayes Confusion Matrix:")
print(cm_nb)

print("Execution time for Multinomial Naive Bayes:", execution_time_nb, "seconds")
execution_time_nb_rounded = round(execution_time_nb)

print("Multinomial Naive Bayes Accuracy : %0.5f \n\n" % accuracy_score(ytest, nb.predict(xtest)))
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(ytest, nb.predict(xtest)))

Modified to remove Hindi stopwords (Dataset 6, Dataset 9, and Dataset 10)

In [ ]:
import pandas as pd
import numpy as np
import re
import time
from stopwordsiso import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

file_path = '{Insert file path here}'
df = pd.read_excel(file_path)

unintended_indices = df[(df['v1'] != 'ham') & (df['v1'] != 'spam')].index
if len(unintended_indices) > 0:
    print("UNINTENDED DATA found at rows:")
    print(df.loc[unintended_indices])

df['v1'] = df['v1'].replace(['ham', 'spam'], [0, 1])

df['Count'] = df['v2'].apply(lambda x: len(str(x)))

corpus = []

for i in range(len(df)):
    msg = df['v2'][i]
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(df['v2'][i]))
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', str(df['v2'][i]))
    msg = re.sub('£|\$', 'moneysymb', str(df['v2'][i]))
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', str(df['v2'][i]))
    msg = re.sub('\d+(\.\d+)?', 'numbr', str(df['v2'][i]))

    msg = re.sub('[^\w\d\s]', ' ', str(df['v2'][i]))

    msg = msg.lower()
    msg = msg.split()

    stopwords_hindi = stopwords("hi")
    msg = [word for word in msg if word not in stopwords_hindi]
    msg = ' '.join(msg)
    corpus.append(msg)

cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

y = df['v1'].astype(str)
le = LabelEncoder()
y = le.fit_transform(y)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=0)

param_grid_dt = {
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3],
    'criterion': ['gini'],
    'max_features': ['sqrt']
}

start_time_dt = time.time()

dt = DecisionTreeClassifier(random_state=50)
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, n_jobs=-1, verbose=1)
grid_search_dt.fit(xtrain, ytrain)

best_dt_params = grid_search_dt.best_params_
print(f"Best Decision Tree Parameters: {best_dt_params}")

dt.fit(xtrain, ytrain)

y_pred_dt = dt.predict(xtest)

end_time_dt = time.time()

execution_time_dt = end_time_dt - start_time_dt
execution_time_dt_rounded = round(execution_time_dt)

cm_dt = confusion_matrix(ytest, y_pred_dt)

print("Decision Tree Confusion Matrix:")
print(cm_dt)

print("Execution time for Decision Tree:", execution_time_dt, "seconds")

print("Decision Tree Accuracy : %0.5f \n\n" % accuracy_score(ytest, dt.predict(xtest)))
print("Decision Tree Classification Report:")
print(classification_report(ytest, dt.predict(xtest)))

param_grid_nb = {
    'alpha': [1, 2, 5]
}

start_time_nb = time.time()

nb = MultinomialNB()

grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, n_jobs=-1, verbose=1)
grid_search_nb.fit(xtrain, ytrain)

best_nb_params = grid_search_nb.best_params_
print(f"Best Naive Bayes Parameters: {best_nb_params}")

nb.fit(xtrain, ytrain)

y_pred_nb = nb.predict(xtest)

end_time_nb = time.time()

execution_time_nb = end_time_nb - start_time_nb

cm_nb = confusion_matrix(ytest, y_pred_nb)

print("\nMultinomial Naive Bayes Confusion Matrix:")
print(cm_nb)

print("Execution time for Multinomial Naive Bayes:", execution_time_nb, "seconds")
execution_time_nb_rounded = round(execution_time_nb)

print("Multinomial Naive Bayes Accuracy : %0.5f \n\n" % accuracy_score(ytest, nb.predict(xtest)))
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(ytest, nb.predict(xtest)))

Modified to remove Persian stopwords (Dataset 7)

In [ ]:
!pip install hazm

In [ ]:
import pandas as pd
import numpy as np
import re
import time
import hazm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from hazm import word_tokenize, stopwords_list
from sklearn.model_selection import GridSearchCV

# Load dataset
file_path = '{Insert file path here}'
df = pd.read_excel(file_path)


unintended_indices = df[(df['v1'] != 'ham') & (df['v1'] != 'spam')].index
if len(unintended_indices) > 0:
    print("UNINTENDED DATA found at rows:")
    print(df.loc[unintended_indices])


df['v1'] = df['v1'].replace(['ham', 'spam'], [0, 1])


df['Count'] = df['v2'].apply(lambda x: len(str(x)))

corpus = []

for i in range(len(df)):
    msg = df['v2'][i]
    msg = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(df['v2'][i]))
    msg = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', str(df['v2'][i]))
    msg = re.sub('£|\$', 'moneysymb', str(df['v2'][i]))
    msg = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', str(df['v2'][i]))
    msg = re.sub('\d+(\.\d+)?', 'numbr', str(df['v2'][i]))

    msg = re.sub('[^\w\d\s]', ' ', str(df['v2'][i]))

    msg = msg.lower()
    msg = msg.split()

    stopwords = stopwords_list()

    msg = [word for word in msg if word not in stopwords]

    msg = ' '.join(msg)
    corpus.append(msg)

cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

y = df['v1'].astype(str)
le = LabelEncoder()
y = le.fit_transform(y)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=0)

param_grid_dt = {
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [2, 3],
    'criterion': ['gini'],
    'max_features': ['sqrt']
}

start_time_dt = time.time()

dt = DecisionTreeClassifier(random_state=50)
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, n_jobs=-1, verbose=1)
grid_search_dt.fit(xtrain, ytrain)

best_dt_params = grid_search_dt.best_params_
print(f"Best Decision Tree Parameters: {best_dt_params}")

dt.fit(xtrain, ytrain)

y_pred_dt = dt.predict(xtest)

end_time_dt = time.time()

execution_time_dt = end_time_dt - start_time_dt
execution_time_dt_rounded = round(execution_time_dt)

cm_dt = confusion_matrix(ytest, y_pred_dt)

print("Decision Tree Confusion Matrix:")
print(cm_dt)

print("Execution time for Decision Tree:", execution_time_dt, "seconds")

print("Decision Tree Accuracy : %0.5f \n\n" % accuracy_score(ytest, dt.predict(xtest)))
print("Decision Tree Classification Report:")
print(classification_report(ytest, dt.predict(xtest)))

param_grid_nb = {
    'alpha': [1, 2, 5]
}

start_time_nb = time.time()

nb = MultinomialNB()

grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, n_jobs=-1, verbose=1)
grid_search_nb.fit(xtrain, ytrain)

best_nb_params = grid_search_nb.best_params_
print(f"Best Naive Bayes Parameters: {best_nb_params}")

nb.fit(xtrain, ytrain)

y_pred_nb = nb.predict(xtest)

end_time_nb = time.time()

execution_time_nb = end_time_nb - start_time_nb

cm_nb = confusion_matrix(ytest, y_pred_nb)

print("\nMultinomial Naive Bayes Confusion Matrix:")
print(cm_nb)

print("Execution time for Multinomial Naive Bayes:", execution_time_nb, "seconds")
execution_time_nb_rounded = round(execution_time_nb)

print("Multinomial Naive Bayes Accuracy : %0.5f \n\n" % accuracy_score(ytest, nb.predict(xtest)))
print("Multinomial Naive Bayes Classification Report:")
print(classification_report(ytest, nb.predict(xtest)))